In [1]:
import os
import sys
import glob
import time
import random
import pandas as pd
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from lib.config import *
from lib.encoder.ffn import FFNEncoder, FFNDecoder
from lib.encoder.vqvae import VQVAE, VectorQuantizer
from lib.train.run_autoencoder_training import AutoencoderTrainer
from lib.utils.pose import get_pose_estimation
import datetime

In [2]:
import os 

os.cpu_count()

8

In [3]:
mp4_files = glob.glob("dataset/corpus/*.mp4")
mp4_files

['dataset/corpus/GEREK _1.mp4',
 'dataset/corpus/I╠çLGINC╠ğ_0.mp4',
 'dataset/corpus/MEMUR_1.mp4',
 'dataset/corpus/BAS╠ğPARMAK_0.mp4',
 'dataset/corpus/YILDIZ_0.mp4',
 'dataset/corpus/ZARF_0.mp4',
 'dataset/corpus/SERVIS_1.mp4',
 'dataset/corpus/HAKLI_0.mp4',
 'dataset/corpus/KIM_1.mp4',
 'dataset/corpus/I╠çKISI_0.mp4',
 'dataset/corpus/DIG╠åER_0.mp4',
 'dataset/corpus/BAYRAK_0.mp4',
 'dataset/corpus/I╠çNS╠ğALLAH_1.mp4',
 'dataset/corpus/HOPARLO╠êR_0.mp4',
 'dataset/corpus/I╠çMZALAMAK_0.mp4',
 'dataset/corpus/GARANTI_0.mp4',
 'dataset/corpus/KUSURA BAKMAMAK_1.mp4',
 'dataset/corpus/SAC╠ğ_0.mp4',
 'dataset/corpus/VERGI _1.mp4',
 'dataset/corpus/YAVAS╠ğLAMAK_0.mp4',
 'dataset/corpus/BAG╠åIMSIZ _0.mp4',
 'dataset/corpus/GO╠êNU╠êL_0.mp4',
 'dataset/corpus/TREN_1.mp4',
 'dataset/corpus/C╠ğAY_1.mp4',
 'dataset/corpus/UC╠ğAK_0.mp4',
 'dataset/corpus/ALET_0.mp4',
 'dataset/corpus/PRATIK_0.mp4',
 'dataset/corpus/I╠çC╠ğIN_0.mp4',
 'dataset/corpus/BIRA_1.mp4',
 'dataset/corpus/

In [4]:
GLOBAL_CONFIG

namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False,
          MEDIAPIPE_MODEL_COMPLEXITY=1,
          MEDIAPIPE_SMOOTH_LANDMARKS=True,
          MEDIAPIPE_ENABLE_SEGMENTATION=False,
          MEDIAPIPE_SMOOTH_SEGMENTATION=True,
          MEDIAPIPE_REFINE_FACE_LANDMARKS=False,
          MEDIAPIPE_MIN_DETECTION_CONFIDENCE=0.5,
          MEDIAPIPE_MIN_TRACKING_CONFIDENCE=0.5,
          MODEL_ENCODER_INPUT_DIM=225,
          MODEL_ENCODER_HIDDEN_DIM=256,
          MODEL_ENCODER_OUTPUT_DIM=768,
          MODEL_VQ_NUM_EMBS=10000,
          MODEL_VQ_EMBED_DIM=768,
          MODEL_VQ_COMMITMENT_COST=0.25,
          MODEL_DECODER_INPUT_DIM=768,
          MODEL_DECODER_HIDDEN_DIM=256,
          MODEL_DECODER_OUTPUT_DIM=225,
          NUM_EPOCHS=100,
          BATCH_SIZE=32,
          LEARNING_RATE=1e-05)

In [5]:
GLOBAL_CONFIG.MEDIAPIPE_MODEL_COMPLEXITY = 2
GLOBAL_CONFIG

namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False,
          MEDIAPIPE_MODEL_COMPLEXITY=2,
          MEDIAPIPE_SMOOTH_LANDMARKS=True,
          MEDIAPIPE_ENABLE_SEGMENTATION=False,
          MEDIAPIPE_SMOOTH_SEGMENTATION=True,
          MEDIAPIPE_REFINE_FACE_LANDMARKS=False,
          MEDIAPIPE_MIN_DETECTION_CONFIDENCE=0.5,
          MEDIAPIPE_MIN_TRACKING_CONFIDENCE=0.5,
          MODEL_ENCODER_INPUT_DIM=225,
          MODEL_ENCODER_HIDDEN_DIM=256,
          MODEL_ENCODER_OUTPUT_DIM=768,
          MODEL_VQ_NUM_EMBS=10000,
          MODEL_VQ_EMBED_DIM=768,
          MODEL_VQ_COMMITMENT_COST=0.25,
          MODEL_DECODER_INPUT_DIM=768,
          MODEL_DECODER_HIDDEN_DIM=256,
          MODEL_DECODER_OUTPUT_DIM=225,
          NUM_EPOCHS=100,
          BATCH_SIZE=32,
          LEARNING_RATE=1e-05)

In [6]:
def get_analyze_statistics(video_name: str) -> list:
    """Returns analyze statistics as list.

    Args:
        video_name (str):

    Returns:
        list: statistics list
            : video_name
            : num_of_frames_total
            : num_of_frames_missing_pose
            : num_of_frames_missing_left_hand
            : num_of_frames_missing_right_hand
            
    """
    
    # print(f"Analyzing: {video_name}")

    estimation, info = get_pose_estimation(video_name, with_info=True)

    pd_info = pd.DataFrame(info, columns=["video_name", "info", "value"])

    num_of_frames_total = pd_info[pd_info['info']
                                  == 'total_number_of_frames']["value"].iloc[0]
    num_of_frames_missing_pose = 0
    num_of_frames_missing_left_hand = 0
    num_of_frames_missing_right_hand = 0

    if "missing_pose" in pd_info["info"].to_list():
        num_of_frames_missing_pose = pd_info.value_counts("info")[
            "missing_pose"]

    if "missing_left_hand" in pd_info["info"].to_list():
        num_of_frames_missing_left_hand = pd_info.value_counts("info")[
            "missing_left_hand"]

    if "missing_right_hand" in pd_info["info"].to_list():
        num_of_frames_missing_right_hand = pd_info.value_counts("info")[
            "missing_right_hand"]

    return [video_name,
            num_of_frames_total,
            num_of_frames_missing_pose,
            num_of_frames_missing_left_hand,
            num_of_frames_missing_right_hand]


In [ ]:
analyze_results = []

for idx, video in enumerate(mp4_files):
    print(f"{idx}/{len(mp4_files)} - Analyzing {video}")
    statistics = get_analyze_statistics(video)
    statistics.append(GLOBAL_CONFIG)
    
    analyze_results.append(statistics)

print(len(analyze_results))

In [13]:
pd_analyze_results = pd.DataFrame(analyze_results,
                                  columns=["video_name", 
                                           "num_of_total_frames", 
                                           "num_of_missing_frames_pose",
                                           "num_of_missing_frames_left_hand",
                                           "num_of_missing_frames_right_hand",
                                           "configuration"
                                           ])
pd_analyze_results

,video_name,num_of_total_frames,num_of_missing_frames_pose,num_of_missing_frames_left_hand,num_of_missing_frames_right_hand,configuration
0,dataset/corpus/GEREK _1.mp4,51,0,51,10,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
1,dataset/corpus/I╠çLGINC╠ğ_0.mp4,56,0,56,30,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
2,dataset/corpus/MEMUR_1.mp4,60,0,60,32,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
3,dataset/corpus/BAS╠ğPARMAK_0.mp4,41,0,41,19,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
4,dataset/corpus/YILDIZ_0.mp4,66,0,21,22,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
...,...,...,...,...,...,...
3390,dataset/corpus/GIZLI_1.mp4,70,0,48,49,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
3391,dataset/corpus/I╠çYI_0.mp4,34,0,34,18,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
3392,dataset/corpus/KULAK_0.mp4,63,0,63,27,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
3393,dataset/corpus/SEBEP_0.mp4,45,0,21,17,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."


In [15]:
formatted_datetime = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Seperator intentionally given as semicolon ";"
# as configuration has comma ","
pd_analyze_results.to_csv(f"analyze/{formatted_datetime}.csv" ,sep=';')

# Optional, write configuration as separate file
# write_global_config_to_file(f"analyze/{formatted_datetime}_config.txt")

# Log Files Analyze

In [133]:
import pandas as pd
import glob 
from types import SimpleNamespace

!pip install tabulate
from tabulate import tabulate

Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [70]:
log_files = glob.glob("analyze/*.csv")

log_analyze = []

for log_file in log_files:
    df_log = pd.read_csv(log_file, sep=";")
    
    len_missing_pose = df_log.sum()["num_of_missing_frames_pose"]
    len_missing_left_hand = df_log.sum()["num_of_missing_frames_left_hand"]
    len_missing_right_hand = df_log.sum()["num_of_missing_frames_right_hand"]
    
    log_analyze.append({
        "name" : log_file,
        "num_of_missing_frames_total" : len_missing_pose + len_missing_left_hand + len_missing_right_hand,
        "num_of_missing_frames_pose" : len_missing_pose,
        "num_of_missing_frames_left_hand"  : len_missing_left_hand,
        "num_of_missing_frames_right_hand" : len_missing_right_hand,
        "configuration" : df_log["configuration"].iloc[0] }
    )

pd_log_analyze = pd.DataFrame(log_analyze)
pd_log_analyze

,name,num_of_missing_frames_total,num_of_missing_frames_pose,num_of_missing_frames_left_hand,num_of_missing_frames_right_hand,configuration
0,analyze/20240402_163040.csv,210885,154,128161,82570,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
1,analyze/20240402_064717.csv,210949,154,128197,82598,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
2,analyze/20240401_204336.csv,211452,154,128180,83118,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
3,analyze/20240401_145215.csv,211467,154,128176,83137,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
4,analyze/20240402_233240.csv,210935,154,128167,82614,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
5,analyze/20240401_073232.csv,211410,154,128136,83120,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
6,analyze/20240401_000331.csv,210874,154,128146,82574,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
7,analyze/20240331_092627.csv,211417,154,128182,83081,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."


In [71]:
pd_log_analyze.sort_values("num_of_missing_frames_total")

,name,num_of_missing_frames_total,num_of_missing_frames_pose,num_of_missing_frames_left_hand,num_of_missing_frames_right_hand,configuration
6,analyze/20240401_000331.csv,210874,154,128146,82574,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
0,analyze/20240402_163040.csv,210885,154,128161,82570,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
4,analyze/20240402_233240.csv,210935,154,128167,82614,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
1,analyze/20240402_064717.csv,210949,154,128197,82598,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
5,analyze/20240401_073232.csv,211410,154,128136,83120,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
7,analyze/20240331_092627.csv,211417,154,128182,83081,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
2,analyze/20240401_204336.csv,211452,154,128180,83118,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."
3,analyze/20240401_145215.csv,211467,154,128176,83137,"namespace(MEDIAPIPE_STATIC_IMAGE_MODE=False, M..."


In [119]:
best_configuration = pd_log_analyze.sort_values("num_of_missing_frames_total").iloc[0]["configuration"]
best_configuration = best_configuration.replace('namespace', '')
best_configuration = best_configuration.replace('(', '')

for item in best_configuration.split(" "):
    if "MEDIAPIPE" in item:
        print(item)

MEDIAPIPE_STATIC_IMAGE_MODE=False,
MEDIAPIPE_MODEL_COMPLEXITY=2,
MEDIAPIPE_SMOOTH_LANDMARKS=True,
MEDIAPIPE_ENABLE_SEGMENTATION=False,
MEDIAPIPE_SMOOTH_SEGMENTATION=True,
MEDIAPIPE_REFINE_FACE_LANDMARKS=False,
MEDIAPIPE_MIN_DETECTION_CONFIDENCE=0.5,
MEDIAPIPE_MIN_TRACKING_CONFIDENCE=0.5,


In [142]:
def get_mediapipe_config(config):
    config = config.replace('namespace', '')
    config = config.replace('(', '')
    
    mediapipe_config = ""
    for item in config.split(" "):
        if "COMPLEXITY" in item or "CONFIDENCE" in item:
            mediapipe_config += item + "\n"

    return mediapipe_config


tabulated_rows = []
for index, row in pd_log_analyze.sort_values("num_of_missing_frames_total").iterrows():

    print_row = []
    headers = ["name", 
               "num_of_missing_frames_total", 
               "num_of_missing_frames_pose", 
               "num_of_missing_frames_left_hand",
               "num_of_missing_frames_right_hand"]
    
    for header in headers:
        print_row.append(row[header])    
    
    print_row.append(get_mediapipe_config(row["configuration"]))
    
    tabulated_rows.append(print_row)
    
headers.append("configuration")
print(tabulate(tabulated_rows , headers=headers ))

name                           num_of_missing_frames_total    num_of_missing_frames_pose    num_of_missing_frames_left_hand    num_of_missing_frames_right_hand  configuration
---------------------------  -----------------------------  ----------------------------  ---------------------------------  ----------------------------------  ----------------------------------------
analyze/20240401_000331.csv                         210874                           154                             128146                               82574  MEDIAPIPE_MODEL_COMPLEXITY=2,
                                                                                                                                                                 MEDIAPIPE_MIN_DETECTION_CONFIDENCE=0.5,
                                                                                                                                                                 MEDIAPIPE_MIN_TRACKING_CONFIDENCE=0.5,
analyze/20240402_163040.csv    

In [49]:
pd_log_analyze.sort_values("num_of_missing_frames_left_hand")

,name,num_of_missing_frames_pose,num_of_missing_frames_left_hand,num_of_missing_frames_right_hand,configuration,total_missing_frames
5,analyze/20240401_073232.csv,154,128136,83120,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...,211410
6,analyze/20240401_000331.csv,154,128146,82574,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...,210874
0,analyze/20240402_163040.csv,154,128161,82570,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...,210885
4,analyze/20240402_233240.csv,154,128167,82614,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...,210935
3,analyze/20240401_145215.csv,154,128176,83137,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...,211467
2,analyze/20240401_204336.csv,154,128180,83118,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...,211452
7,analyze/20240331_092627.csv,154,128182,83081,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...,211417
1,analyze/20240402_064717.csv,154,128197,82598,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...,210949


In [47]:
pd_log_analyze.sort_values("num_of_missing_frames_right_hand")

,name,num_of_missing_frames_pose,num_of_missing_frames_left_hand,num_of_missing_frames_right_hand,configuration
0,analyze/20240402_163040.csv,154,128161,82570,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...
6,analyze/20240401_000331.csv,154,128146,82574,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...
1,analyze/20240402_064717.csv,154,128197,82598,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...
4,analyze/20240402_233240.csv,154,128167,82614,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...
7,analyze/20240331_092627.csv,154,128182,83081,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...
2,analyze/20240401_204336.csv,154,128180,83118,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...
5,analyze/20240401_073232.csv,154,128136,83120,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...
3,analyze/20240401_145215.csv,154,128176,83137,0 namespace(MEDIAPIPE_STATIC_IMAGE_MODE=...
